# Slowly Changing Dimension Type 2 (SCD2) - User Table

## Overview
This notebook implements an SCD2 dimension table for users (`dimuser`). The goal is to track changes in user details over time.

## Steps
1. **Load existing user dimension table (`dimuser`)** from Delta Lake.
2. **Load latest changes** from the operational database (`velo_users`).
3. **Detect new and changed records** using an MD5 hash comparison.
4. **Insert new records** while keeping old versions with history.
5. **Use Delta Lake's `MERGE INTO`** for efficient updates.

## Key Fields
| Column     | Description |
|------------|------------|
| `user_sk`  | Unique identifier for each user record |
| `userid`   | Business key (same across historical records) |
| `street`, `city`, etc. | Address details |
| `md5`      | Hash of address details to detect changes |
| `scd_start` | Start of record validity |
| `scd_end`   | End of record validity |
| `current`   | Indicates active record (TRUE) or historical (FALSE) |

## Example
If a user changes address, a new row is added while the old one is kept with an end date.


### 📌 Explanation
✅ **Import required libraries:**
- `pyspark.sql` → Provides the Spark DataFrame API.
- `pyspark.sql.functions` → Contains useful SQL functions.
- `ConnectionConfig (cc)` → Custom module to set up connections.

✅ **Set up the environment:**
- `cc.setupEnvironment()` → Configures the Spark environment.
- `cc.listEnvironment()` → Lists current environment settings.


In [ ]:
import ConnectionConfig as cc
cc.setupEnvironment()
cc.listEnvironment()

### 📌 Explanation
✅ **Start a local Spark cluster:**
- `"dimUserChanges"` → Name of the cluster.
- `4` → Number of worker threads.

✅ **Get the active Spark session:**
- `getActiveSession()` → Ensures the session is running.

In [ ]:
spark = cc.startLocalCluster("dimUserChanges",4)
spark.getActiveSession()

### 📌 Explanation
✅ **Capture the job execution timestamp:**
- `datetime.now()` → Retrieves the current timestamp.
- This timestamp will be used to track when records were processed.

In [ ]:
from datetime import *
run_timestamp =datetime.now() #The job runtime is stored in a variable
print(run_timestamp)

### 📌 Explanation
✅ **Load the existing dimension table:**
- `DeltaTable.forPath(...)` → Loads the `dimuser` table from Delta Lake.

✅ **Create a temporary SQL view:**
- `createOrReplaceTempView("dim_users_current")` → Allows querying via SQL.

✅ **Show existing records:**
- `spark.sql("SELECT * FROM dim_users_current").show()` → Displays data.


In [ ]:
from delta import DeltaTable
current_user_table = DeltaTable.forPath(spark, "./spark-warehouse/dimuser")

In [ ]:
current_user_table.toDF().createOrReplaceTempView("dim_users_current")

In [ ]:
spark.sql("select * from dim_users_current").show()

### 📌 Explanation
✅ **Read new user data from the operational database:**
- Uses **JDBC** to connect to the database.

✅ **Create a temporary SQL view:**
- `createOrReplaceTempView("users_operational_db")` → Enables SQL queries on the latest user data.


In [ ]:
### LOAD THE NEWEST CHANGES FROM THE OPERATIONAL DATABASE
df_users = spark.read \
    .format("jdbc") \
    .option("driver", cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "velo_users") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .load()

df_users.createOrReplaceTempView("users_operational_db")

### 📌 Explanation
✅ **Transform the operational data into a dimension format:**
- `uuid()` → Generates a **unique key** for each record.
- `md5(...)` → Creates a **hash of address details** to detect changes.

✅ **Create a SQL view for transformed data:**
- `createOrReplaceTempView("dim_users_new")` → Allows comparison with existing data.

✅ **Show transformed records:**
- `dim_users_new.show()` → Displays the new data.


In [ ]:
### TRANSFORM THE SOURCE TABLE TO the dimension format
### IMPORTANT !!!!!!!!! CHECKING ONLY THE STREET FOR NOW !!!!!!!!
dim_users_new = spark.sql( "select uuid() as source_user_sk, \
                                        userid as source_userid, \
                                        name as source_name, \
                                        street as source_street, \
                                        md5(concat( street, number, zipcode, city, country_code)) as source_md5 \
                                    from users_operational")
dim_users_new.createOrReplaceTempView("dim_users_new")

In [ ]:
dim_users_new.show()

### 📌 Explanation
✅ **Detect changes:**
- `LEFT OUTER JOIN` → Compare new records (`dim_users_new`) with the existing ones (`dim_users_current`).
- `WHERE dwh.userid IS NULL` → Identifies **new users**.
- `OR dwh.md5 <> source.source_md5` → Identifies **modified users**.


In [ ]:
detectedChanges=spark.sql(f"select * \
                          from dim_users_new as source \
                          left outer join dim_users_current as dwh on dwh.userid == source.source_userid and dwh.current == true \
                          where dwh.userid is null or dwh.md5 <> source.source_md5")

detectedChanges.createOrReplaceTempView("detectedChanges")

In [ ]:
#DEBUG CODE TO SHOW CONTENT OF DETECTED CHANGES
detectedChanges.show()

### 📌 Explanation
✅ **Prepare data for update and insert:**
- Inserts **new records**.
- Updates **existing records** (sets `scd_end` and `current = FALSE`).


In [ ]:
upserts = spark.sql(f"select source_user_sk as user_sk,\
                                source_userid as userid,\
                                source_name as name,\
                                source_street as street,\
                                to_timestamp('{run_timestamp}') as scd_start, \
                                to_timestamp('2100-12-12','yyyy-MM-dd') as scd_end,\
                                source_md5 as md5,\
                                true as current\
                        from  detectedChanges\
                        union \
                        select  userSK,\
                                userid,\
                                name,\
                                street,\
                                scd_start,\
                                to_timestamp('{run_timestamp}') as scd_end,\
                                md5, \
                                false \
                                from detectedChanges \
                        where current is not null")

upserts.createOrReplaceTempView("upserts")

In [ ]:
#DEBUG CODE TO SHOW CONTENT OF UPSERTS
spark.sql("select * from upserts").show()

### 📌 Explanation
✅ **Perform the SCD2 merge operation:**
- **UPDATE** existing records → Sets `scd_end` and `current = FALSE`.
- **INSERT** new records → Tracks changes as new entries.


In [ ]:
spark.sql("MERGE INTO dim_users_current AS target \
          using upserts AS source ON target.userid = source.userid and source.current = false and target.current=true \
          WHEN MATCHED THEN UPDATE SET scd_end = source.scd_end, current = source.current  \
          WHEN NOT MATCHED THEN INSERT (userSK, userid, name, street, scd_start, scd_end, md5, current) values (source.user_sk, source.userid, source.name, source.street, source.scd_start, source.scd_end, source.md5, source.current)")


### 📌 Explanation
✅ **Display the final dimension table sorted by `userid` and `scd_start`**  
- Shows **historical versions** and the **latest active record**.


In [ ]:
current_user_table.toDF().sort("userid", "scd_start").show(100)

In [ ]:
spark.stop()